In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import pickle

In [10]:
with open('train_images.pkl', 'rb') as file:
    images= pickle.load(file)

In [11]:
with open('train_masks.pkl', 'rb') as file:
    masks=pickle.load(file)

In [12]:
with open('val_images.pkl', 'rb') as file:
    val_images=pickle.load(file)

In [13]:
with open('val_masks.pkl', 'rb') as file:
    val_masks=pickle.load(file)

In [14]:
images.shape

(137, 256, 256, 3)

In [15]:
masks.shape

(137, 256, 256, 1)

In [16]:
val_images.shape

(4, 256, 256, 3)

In [17]:
val_masks.shape

(4, 256, 256, 1)

In [18]:
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.keras.activations.sigmoid(y_pred)

    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - (numerator / (denominator + tf.keras.backend.epsilon()))

In [19]:
def identity_block(X, filters):
    f1, f2, f3 = filters
    X_copy = X

    # 1st Layer
    X = layers.Conv2D(filters=f1, kernel_size=1, padding="valid")(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)

    # 2nd Layer
    X = layers.Conv2D(filters=f2, kernel_size=3, padding="same")(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)

    # 3rd Layer
    X = layers.Conv2D(filters=f3, kernel_size=1, padding="valid")(X)
    X = layers.BatchNormalization()(X)

    # Skip Connection
    X = layers.Add()([X, X_copy])
    X = layers.ReLU()(X)

    return X


In [20]:
def conv_block(X, filters, s=2):
    f1, f2, f3 = filters
    X_copy = X

    # 1st Layer
    X = layers.Conv2D(filters=f1, kernel_size=1, strides=s, padding="valid")(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)

    # 2nd Layer
    X = layers.Conv2D(filters=f2, kernel_size=3, padding="same")(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)

    # 3rd Layer
    X = layers.Conv2D(filters=f3, kernel_size=1, padding="valid")(X)
    X = layers.BatchNormalization()(X)

    # Adjust Skip Connection Dimensions
    X_copy = layers.Conv2D(filters=f3, kernel_size=1, strides=s, padding="valid")(X_copy)
    X_copy = layers.BatchNormalization()(X_copy)

    # Skip Connection
    X = layers.Add()([X, X_copy])
    X = layers.ReLU()(X)

    return X

In [21]:
def resnet50_encoder(input_shape):
    X_input = layers.Input(input_shape)

    # Initial Conv Layer
    X = layers.ZeroPadding2D((3, 3))(X_input)
    X = layers.Conv2D(64, (7, 7), strides=2)(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)
    X = layers.MaxPooling2D((3, 3), strides=2)(X)

    # ResNet Stages (with Skip Connections)
    enc1 = conv_block(X, filters=[64, 64, 256], s=1) #Here we got 64,64,64 X shape, now what i will do i will call conv_block..because in skip connection
                                                     #we need to balance the feature map...
    enc1 = identity_block(enc1, filters=[64, 64, 256])#completed again final size is 64,64,256
    enc1 = identity_block(enc1, filters=[64, 64, 256])#again 64,64,256

    enc2 = conv_block(enc1, filters=[128, 128, 512], s=2) #after succefully complete this, size become 32,32,512
    enc2 = identity_block(enc2, filters=[128, 128, 512])#completed ,and final size is 32,32,512
    enc2 = identity_block(enc2, filters=[128, 128, 512])
    enc2 = identity_block(enc2, filters=[128, 128, 512])#final size become 32,32,512

    enc3 = conv_block(enc2, filters=[256, 256, 1024], s=2) #completed and size is 16,16,1024
    enc3 = identity_block(enc3, filters=[256, 256, 1024])#completed..
    enc3 = identity_block(enc3, filters=[256, 256, 1024])
    enc3 = identity_block(enc3, filters=[256, 256, 1024])
    enc3 = identity_block(enc3, filters=[256, 256, 1024])
    enc3 = identity_block(enc3, filters=[256, 256, 1024])#completed and final shape is 16,16,1024

    enc4 = conv_block(enc3, filters=[512, 512, 2048], s=2)#completed and shape become 8,8,2048
    enc4 = identity_block(enc4, filters=[512, 512, 2048])
    enc4 = identity_block(enc4, filters=[512, 512, 2048])#final shape is 8,8,2048..so tough

    # Bottleneck (8x8)
    bottleneck = layers.Conv2D(2048, kernel_size=3, padding="same", activation="relu")(enc4)#this part is interesting..we again put the input into
                                                                                            #convolutional operation and size remain same 
                                                                                            # 8,8,2024

    return Model(inputs=X_input, outputs=[bottleneck, enc4, enc3, enc2, enc1])#so now for my decoder part, and i need to use attention module,
                                                                              #i need enc1,enc2,enc3,enc4 and bottlenect

In [22]:
def attention_gate(skip, gating):
    skip_conv = layers.Conv2D(gating.shape[-1], kernel_size=1, padding="same")(skip)
    gate_conv = layers.Conv2D(gating.shape[-1], kernel_size=1, padding="same")(gating)

    attention = layers.Add()([skip_conv, gate_conv])
    attention = layers.ReLU()(attention)
    attention_weights = layers.Conv2D(1, kernel_size=1, activation="sigmoid", padding="same")(attention)

    return layers.Multiply()([skip, attention_weights])

In [23]:
def decoder_block(x, skip, filters):
   
    x = layers.Conv2DTranspose(filters, kernel_size=2, strides=2, padding="same")(x) #upsampled

    target_size = (tf.keras.backend.int_shape(x)[1], tf.keras.backend.int_shape(x)[2])

    skip = layers.Resizing(target_size[0], target_size[1])(skip) #here we get resize feature map as x
    skip = attention_gate(skip, x)# 

    x = layers.Concatenate()([x, skip])  
    x = layers.Conv2D(filters, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.Conv2D(filters, kernel_size=3, padding="same", activation="relu")(x)

    return x

In [24]:
def unet_with_manual_resnet50(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    resnet_encoder = resnet50_encoder(input_shape)
    bottleneck, enc4, enc3, enc2, enc1 = resnet_encoder(inputs)

    x = decoder_block(bottleneck, enc4, 1024) #calling with size of bottleneck is 8,8,2024 en4 is 8,8,2048
    x = decoder_block(x, enc3, 512)  
    x = decoder_block(x, enc2, 256)  
    x = decoder_block(x, enc1, 128) 

   
    x = layers.Conv2DTranspose(64, kernel_size=2, strides=2, padding="same")(x)  
    x = layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(x)

    
    outputs = layers.Conv2D(num_classes, kernel_size=1, activation="sigmoid")(x) 

    return Model(inputs, outputs)

In [25]:
model = unet_with_manual_resnet50(input_shape=(256, 256, 3), num_classes=1)

In [26]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional (Functional)       │ [(None, 8, 8, 2048),      │      61,338,496 │ input_layer[0][0]          │
│                               │ (None, 8, 8, 2048),       │                 │                            │
│                               │ (None, 16, 16, 1024),     │                 │                            │
│                               │ (None, 32, 32, 512),      │                 │                            │
│                               │ (None, 63, 63, 256)]      │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose              │ (None, 16, 16, 1024)      │       8,389,632 │ functional[0][0]           │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ resizing (Resizing)           │ (None, 16, 16, 2048)      │               0 │ functional[0][1]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_54 (Conv2D)            │ (None, 16, 16, 1024)      │       2,098,176 │ resizing[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_55 (Conv2D)            │ (None, 16, 16, 1024)      │       1,049,600 │ conv2d_transpose[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_16 (Add)                  │ (None, 16, 16, 1024)      │               0 │ conv2d_54[0][0],           │
│                               │                           │                 │ conv2d_55[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_49 (ReLU)               │ (None, 16, 16, 1024)      │               0 │ add_16[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_56 (Conv2D)            │ (None, 16, 16, 1)         │           1,025 │ re_lu_49[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply (Multiply)           │ (None, 16, 16, 2048)      │               0 │ resizing[0][0],            │
│                               │                           │                 │ conv2d_56[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 16, 16, 3072)      │               0 │ conv2d_transpose[0][0],    │
│                               │                           │                 │ multiply[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_57 (Conv2D)            │ (None, 16, 16, 1024)      │      28,312,576 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_58 (Conv2D)            │ (None, 16, 16, 1024)      │       9,438,208 │ conv2d_57[0][0]            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 126,910,853 (484.13 MB)

 Trainable params: 126,857,733 (483.92 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=dice_loss,
    metrics=['accuracy']
)

In [28]:
EPOCHS=25 
BATCH_SIZE = 16

In [29]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [31]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),  
    ModelCheckpoint('best_unet_model.keras', save_best_only=True, monitor='val_loss', mode='min')  
]

In [32]:
history = model.fit(
    images, masks,
    validation_data=(val_images, val_masks),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks
)


Epoch 1/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 665s 72s/step - accuracy: 0.6842 - loss: 0.7793 - val_accuracy: 0.1507 - val_loss: 0.8149
Epoch 2/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 586s 65s/step - accuracy: 0.5675 - loss: 0.7814 - val_accuracy: 0.2960 - val_loss: 0.8144
Epoch 3/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 570s 60s/step - accuracy: 0.4214 - loss: 0.7676 - val_accuracy: 0.8913 - val_loss: 0.8152
Epoch 4/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 543s 58s/step - accuracy: 0.4597 - loss: 0.7588 - val_accuracy: 0.8913 - val_loss: 0.8186
Epoch 5/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 541s 55s/step - accuracy: 0.5170 - loss: 0.7523 - val_accuracy: 0.8913 - val_loss: 0.8203
Epoch 6/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 485s 53s/step - accuracy: 0.5630 - loss: 0.7628 - val_accuracy: 0.8913 - val_loss: 0.8211
Epoch 7/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 2025s 244s/step - accuracy: 0.5381 - loss: 0.7588 - val_accuracy: 0.8913 - val_loss: 0.8213


In [33]:
with open('test_images.pkl', 'rb') as file:
    test_images= pickle.load(file)

In [34]:
with open('test_masks.pkl', 'rb') as file:
    test_masks= pickle.load(file)

In [35]:
from tensorflow.keras.models import load_model